# Initialization of the notebook

## Install the package

First, you need to install the package from github and import it:

In [1]:
# Install the bulkhours package
!rm -rf bulkhours && git clone https://github.com/guydegnol/bulkhours.git

# Import the bulkhours package
import bulkhours

# Import needed libraries
import numpy as np

# Generate header links (only for the demo)
bulkhours.generate_header_links("examples/1_Simple_Exercice_Automation.ipynb")

Cloning into 'bulkhours'...
remote: Enumerating objects: 5909, done.
remote: Counting objects: 100% (1508/1508), done.
remote: Compressing objects: 100% (553/553), done.
remote: Total 5909 (delta 1022), reused 1389 (delta 909), pack-reused 4401
Receiving objects: 100% (5909/5909), 42.66 MiB | 18.48 MiB/s, done.
Resolving deltas: 100% (3859/3859), done.


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/guydegnol/bulkhours/blob/main/examples/1_Simple_Exercice_Automation.ipynb) [![GitHub](https://badgen.net/badge/icon/Open%20in%20Github?icon=github&label)](https://github.com/guydegnol/bulkhours/blob/main/examples/1_Simple_Exercice_Automation.ipynb) [![Open in AWS Studio](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/guydegnol/bulkhours/blob/main/examples/1_Simple_Exercice_Automation.ipynb) [![Open in Visual Studio Code](https://img.shields.io/static/v1?logo=visualstudiocode&label=&message=Open%20in%20Visual%20Studio&labelColor=2c2c32&color=007acc&logoColor=007acc)](https://vscode.dev/github/guydegnol/bulkhours/blob/main/examples/1_Simple_Exercice_Automation.ipynb) [![CC-by-nc-sa license](https://badgen.net/badge/icon/CC%20by-nc-sa?label=Licence)](https://creativecommons.org/licenses/by-nc-sa/4.0) 


## Connection to the evaluation database

To activate the evaluation tools, you need to identify yourself on the database server.


In [2]:
# You need to identify yourself first to activate the evaluation tools
bulkhours.init_env(email="luke.skywalker@jedi.com", database=None)

Import BULK Helper cOURSe (version='5.2.29'🚀'):⚠️Database is local (security_level=0). Export your config file if you need persistency.⚠️
(db, subject, virtual_room, nb_id, user) = (starwars.json, s, light_side, n, luke.skywalker@jedi.com✅)


* If the database `argument` is not set up, it will simply be the following local file:
```python
# This file will be loaded from the bulkhours package directory
database ="data/cache/starwars.json"
```

* You can configure it with your own server:
```python
# This file will be loaded from the server absolute path
database ="/content/mydatabase.json"
```

* It can also be a database config information:
```python
# This config file is only compatible with a google firestore server for now.
database = {
    "type": "service_account",
    ...,
    "universe_domain": "googleapis.com"
    }
```

* Another solution is to ask a token to bulkhours@guydegnol.net . With this token, you should have access to a secured database (with id check and separated rights between students and admins), api to openai, huggingface, etc...
```python
database = "bkloud@SUBJECT/teacher::eNq-XXXXXXXXXXXHXXXXX-L29tB"
database = "bkloud@SUBJECT/CLASSROOM_student::eNq-XXXXXXXXXXXHXXXXX-9tB"
```


>[💡Documentation: 📱**`bulkhours.init_env`**](https://github.com/guydegnol/bulkhours/wiki/init_env)



# Create an exercice with automatic evaluation

Here is the exercice I want to give to my students:

> **Exercise**: Using `numpy.exp`, implement the $\sigma(z) = \frac{1}{1 + e^{-z}}$ to make predictions.

The expected solution is the following:

```python
def sigmoid(z):
  s = 1 / (1 + np.exp(-z))
  return s
```

## Define an exercice id

To identify an exercice, we need to define an id for each cell (aka `cell_id`). We can do it by using the `%%evaluation_cell_id` magic method.

For instance, here is the content of a cell that a student might see:

In [3]:
%%evaluation_cell_id -i my_first_exercice

def sigmoid(z):
  # Implement the sigmoid function
  s = z # ...
  return s

for z in [1, 3, 7]:
  print(f"sigmoid({z})={sigmoid(z)}")

Output()

The id of the exercice has been chosen as `my_first_exercice`.
With this identification, we should ne able to exchange information from one notebook to another.

**As a student, you have by defaults 2 available buttons:**
* One to submit the cell content as an answer,
* Another one to get the solution when it is available.

>[💡Documentation: 🔗**`evaluation_cell_id`**](https://github.com/guydegnol/bulkhours/wiki/evaluation_cell_id)

## Teacher mode versus student mode

Normaly, all the following code should be runned in a teacher notebook.
In demo mode, you can simply switch to the teacher mode calling the
```bulkhours.init_env``` with the ```is_admin=True``` argument.


In [4]:
bulkhours.init_env(email="yoda@jedi.com", is_admin=True)

Import BULK Helper cOURSe (version='5.2.29'🚀', ⚠️ in admin/teacher🎓 mode⚠️):⚠️Database is local (security_level=0). Export your config file if you need persistency.⚠️
(db, subject, virtual_room, nb_id, user) = (starwars.json, s, light_side, n, yoda@jedi.com🎓)


## Accessing someone else notebook cell


As an admin of the project, you can access the cell content of another user like that:

In [5]:
bulkhours.admin.evaluate("my_first_exercice", user="luke.skywalker@jedi.com")

HTML(value="<b><font face='FiraCode Nerd Font' size=4 color='red'>Code (Luke.s)📚<font></b>", layout=Layout(hei…

def sigmoid(z):
  # Implement the sigmoid function
  s = z # ...
  return s

for z in [1, 3, 7]:
  print(f"sigmoid({z})={sigmoid(z)}")

HTML(value="<b><font face='FiraCode Nerd Font' size=4 color='red'>Execution (Luke.s)💻<font></b>", layout=Layou…

sigmoid(1)=1
sigmoid(3)=3
sigmoid(7)=7


Output()

You also have the possibility to directly evaluate the work of a student.

>[💡Documentation: ✏️**`bulkhours.admin.evaluate`**](https://github.com/guydegnol/bulkhours/wiki/evaluate)


## Definition of a criterion for automatic correction


Now, we want to have an automatic way of evaluate one student, for several data points. You can dot it like this:


In [6]:
%%evaluation_cell_id -i my_first_exercice
import numpy as np

def sigmoid(z):
  s = 1 / (1 + np.exp(-z))
  return s

for z in [1, 3, 7]:
  print(f"sigmoid({z})={sigmoid(z)}")

def student_evaluation_function():
    score = 0
    score += bulkhours.is_equal(student.sigmoid(np.array([1, 3, 7])), max_score=5)
    score += 2.5 * (student.sigmoid(3) == teacher.sigmoid(3))
    score += bulkhours.is_equal(student.sigmoid(4), teacher.sigmoid(4), max_score=2.5)
    return score

Output()


Executing the previous cell, **as a teacher, you have 4 available buttons:**
* One to submit the cell content as an answer (student mode),
* Another one to get the solution when it is available (student mode),
* One to publush the current cell as the answer (student_evaluation_function is ignored) (teacher mode),
* Another to evaluate all students who have submitted answers (teacher mode).

**Methods used:**
* `student_evaluation_function`
> When found in a cell, the `student_evaluation_function` is used to perform the automatic evaluation. It returns the student's 'score'.
>
>[💡Documentation: ❓**`student_evaluation_function`**](https://github.com/guydegnol/bulkhours/wiki/student_evaluation_function)


* `bulkhours.is_equal`
> This method compares the students results data with reference data (by default, the teacher's data).
>
>[💡Documentation: 💻**`bulkhours.is_equal`**](https://github.com/guydegnol/bulkhours/wiki/is_equal)
